In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "HyDE"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

Environment Variables Set:
LANGSMITH_TRACING: true
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: HyDE


In [4]:
import langchain
import sys
import pandas as pd
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.retrievers import BaseRetriever
from langchain_community.document_loaders import PyPDFLoader
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=4000,
)

In [7]:
# 1. Load embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_vector_store = FAISS.load_local(
    "nvidia_large_baseline", 
    embeddings,
    allow_dangerous_deserialization=True
)

## Setting up the HyDE Retriever

In [6]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS

@traceable
class HyDERetriever:
    def __init__(self, vector_store, embeddings, chunk_size=800):
        # Use the passed vector store and embeddings
        self.vectorstore = vector_store
        self.embeddings = embeddings
        
        # Initialize LLM for hypothesis generation
        self.llm = llm
        # Set chunk size for hypothesis document
        self.chunk_size = chunk_size
        
        # Create Hyde prompt template
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            The document size should be approximately {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm
        
    def generate_hypothetical_document(self, query):
        """Generate a hypothetical document for the given query."""
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    def retrieve_with_score(self, query, k=4):
        """Retrieve documents using Hypothetical Document Expansion (HyDE)."""
        # Generate hypothetical document
        hypothetical_doc = self.generate_hypothetical_document(query)
        
        # Perform similarity search using the hypothetical document
        similar_docs =  self.vectorstore.similarity_search_with_score(hypothetical_doc, k=k)
        
        return similar_docs, hypothetical_doc

In [8]:
# Creating instance 
hyde_retriever = HyDERetriever(
    vector_store=new_vector_store, 
    embeddings=embeddings
)

# Defining query example
query = "Who is the CEO fo NVIDIA?"

## Experiment

In [10]:
retrieved_docs, hypothetical_doc = hyde_retriever.retrieve_with_score(query)

# To see what's behind the curtains 
print("Hypothetical Document:")
print(hypothetical_doc)
print("\nRetrieved Documents:")
for doc, score in retrieved_docs:
    print("\n---\n", doc.page_content)


Hypothetical Document:
**Document Title: Current CEO of NVIDIA**

**Date: October 2023**

**Introduction:**
NVIDIA Corporation, a leading technology company known for its advancements in graphics processing units (GPUs) and artificial intelligence (AI), has been under the leadership of Jensen Huang since its inception in 1993. As of October 2023, Jensen Huang continues to serve as the President and Chief Executive Officer (CEO) of NVIDIA.

**Background:**
Jensen Huang co-founded NVIDIA and has been instrumental in its growth and innovation. Under his leadership, the company has expanded its product offerings beyond gaming into sectors such as data centers, automotive technology, and AI. Huang's vision has positioned NVIDIA as a key player in the tech industry, particularly in the realms of machine learning and deep learning.

**Achievements:**
Huang has received numerous accolades for his contributions to technology and business, including being named to Fortune's list of the World's 5

## RAG pipeline setup

In [11]:
def setup_rag_pipeline():
    """
    Creates a chain (or pipeline) that:
    1) Takes {"input": question, "context": retrieved_docs}
    2) Formats them into a prompt
    3) Sends to an LLM
    4) Returns the generated text as a string
    """
    template = """You are a helpful assistant that answers questions based on the provided context.
Use the provided context to answer the question.

Question: {input}
Context: {context}
Answer:
"""
    # The ChatPromptTemplate can build a system/human style message,
    # but here we simply have a single message with placeholders.
    prompt = ChatPromptTemplate.from_template(template)

    # Build a simple pipeline:
    #   - The "inputs" to the pipeline are { context, input } from RunnablePassthrough
    #   - They feed into the prompt template
    #   - Then pass the result into llm
    #   - Then parse the output to a string
    rag_pipeline = (
        {"context": RunnablePassthrough(), "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_pipeline

In [12]:
@traceable
def query_rag(
    question: str,
    hyde_retriever: HyDERetriever,
    top_k: int = 4
):
    """
    1) Uses HyDERetriever to generate a hypothetical document,
       then retrieves top_k documents with scores.
    2) Displays them in a DataFrame (descending by score).
    3) Feeds the user 'question' + combined 'context' into the RAG pipeline.
    4) Returns final answer, contexts, and the hypothetical_doc.
    """

    # Step 1: Retrieve docs (and scores) with HyDE
    retrieved_docs_with_scores, hypothetical_doc = hyde_retriever.retrieve_with_score(
        question, k=top_k
    )

    # Sort the retrieved docs by descending score
    retrieved_docs_with_scores.sort(key=lambda x: x[1], reverse=True)

    # Step 2: Display them in a DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),
            "Retrieved Chunk": doc.page_content[:1000] + "..."
        }
        for doc, score in retrieved_docs_with_scores
    ])

    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.width", 1000)

    display(df)

    # Extract just the text of each retrieved chunk
    retrieved_contexts = [doc.page_content for doc, _ in retrieved_docs_with_scores]

    # Combine them into one big string for the final prompt (or you can be more selective).
    combined_context = "\n\n".join(retrieved_contexts)

    # Step 3: Run the RAG pipeline
    # The pipeline expects {"input": question, "context": docs}
    answer = rag_chain.invoke({"input": question, "context": combined_context})

    # Step 4: Return final answer, the raw contexts, plus hypothetical doc
    return {
        "answer": answer,
        "contexts": retrieved_contexts,
        "hypothetical_doc": hypothetical_doc
    }


In [13]:
# Instantiate our RAG pipeline (once).
rag_chain = setup_rag_pipeline()

In [14]:
question = "Who is the CEO of NVIDIA?"

In [15]:
result = query_rag(question, hyde_retriever, top_k=4)

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,0.8985,"“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request...."
1,NVIDIA_Report_2024.pdf,3,0.8983,"marking the “Big Bang” moment of AI. We introduced our first Tensor Core GPU in 2017, built from the ground-up for the new era of AI, and our first autonomousdriving system-on-chips, or SoC, in 2018. Our acquisition of Mellanox in 2020 expanded our innovation canvas to include networking and led to the introductionof a new processor class – the data processing unit, or DPU. Over the past 5 years, we have built full software stacks that run on top of our GPUs and CUDA to\nbring AI to the world’s largest industries, including NVIDIA DRIVE stack for autonomous driving, Clara for healthcare, and Omniverse for industrial digitalization;and introduced the NVIDIA AI Enterprise software – essentially an operating system for enterprise AI applications. In 2023, we introduced our first data center..."
2,NVIDIA_Report_2024.pdf,11,0.6146,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.4684,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


In [16]:
result["answer"]

'The CEO of NVIDIA is Jen-Hsun Huang.'

In [17]:
len(result["contexts"])

4

## RAG Evaluation Dataset

In [18]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [22]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_3811/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [23]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"], hyde_retriever, top_k=4)
    return {"answer": response["answer"]}

In [24]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"], hyde_retriever, top_k=4)
    return {"answer": response["answer"], "contexts": response["contexts"]}

### RAG Answer VS Reference prompt

In [25]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [26]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [30]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="HyDE",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'HyDE-c0124027' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=7b1182e9-8cbc-43ee-a12c-2e55d80bf19d




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,33,0.9129,"Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nRecent Developments, Future Objectives and Challenges\nDemand and Supply, Product Transitions, and New Products and Business Models\nDemand for our data center systems and products surged in fiscal year 2024. Entering fiscal year 2025, we are gathering customer demand indications acrossseveral product transitions. We have demand visibility for our new data center products ramping later in fiscal year 2025. We have increased our supply andcapacity purchases with existing suppliers, added new vendors and entered into prepaid manufacturing and capacity agreements. These increased purchase..."
1,NVIDIA_Report_2024.pdf,33,0.8840,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
2,NVIDIA_Report_2024.pdf,11,0.6498,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.4616,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.5782,"Spectrum-X, an accelerated networking platform for AI.\nGaming revenue for fiscal year 2024 was $10.4 billion, up 15% from fiscal year 2023. In Gaming, we launched the GeForce RTX 4060 and 4070 GPUs based onthe NVIDIA Ada Lovelace architecture. We announced NVIDIA Avatar Cloud Engine for Games, a custom AI model foundry service using AI-powered natural\nlanguage interactions to transform games and launched DLSS 3.5 Ray Reconstruction. Additionally, we released TensorRT-LLM for Windows and launchedGeForce RTX 40-Series SUPER GPUs. Gaming reached a milestone of 500 AI-powered RTX games and applications utilizing NVIDIA DLSS, ray tracing and\nother NVIDIA RTX technologies...."
1,NVIDIA_Report_2024.pdf,35,0.5465,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,35,0.4462,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
3,NVIDIA_Report_2024.pdf,35,0.3502,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6059,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators...."
1,NVIDIA_Report_2024.pdf,6,0.5950,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,3,0.5674,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
3,NVIDIA_Report_2024.pdf,4,0.4473,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."


### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [27]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [28]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [33]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="HyDE",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'HyDE-47e8b97f' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=7ece57cd-aa30-4c31-ba08-d9f7a2cc6f6f




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6061,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators...."
1,NVIDIA_Report_2024.pdf,6,0.5949,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,3,0.5673,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
3,NVIDIA_Report_2024.pdf,4,0.4475,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.5735,"Spectrum-X, an accelerated networking platform for AI.\nGaming revenue for fiscal year 2024 was $10.4 billion, up 15% from fiscal year 2023. In Gaming, we launched the GeForce RTX 4060 and 4070 GPUs based onthe NVIDIA Ada Lovelace architecture. We announced NVIDIA Avatar Cloud Engine for Games, a custom AI model foundry service using AI-powered natural\nlanguage interactions to transform games and launched DLSS 3.5 Ray Reconstruction. Additionally, we released TensorRT-LLM for Windows and launchedGeForce RTX 40-Series SUPER GPUs. Gaming reached a milestone of 500 AI-powered RTX games and applications utilizing NVIDIA DLSS, ray tracing and\nother NVIDIA RTX technologies...."
1,NVIDIA_Report_2024.pdf,35,0.5705,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,35,0.4695,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
3,NVIDIA_Report_2024.pdf,35,0.3598,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,33,0.9133,"Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nRecent Developments, Future Objectives and Challenges\nDemand and Supply, Product Transitions, and New Products and Business Models\nDemand for our data center systems and products surged in fiscal year 2024. Entering fiscal year 2025, we are gathering customer demand indications acrossseveral product transitions. We have demand visibility for our new data center products ramping later in fiscal year 2025. We have increased our supply andcapacity purchases with existing suppliers, added new vendors and entered into prepaid manufacturing and capacity agreements. These increased purchase..."
1,NVIDIA_Report_2024.pdf,33,0.8842,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
2,NVIDIA_Report_2024.pdf,11,0.6502,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.4613,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [29]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [30]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [37]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="HyDE",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'HyDE-6980195d' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=5b153382-a655-4e13-8f65-4a3e8306bd09




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,3,0.8880,"marking the “Big Bang” moment of AI. We introduced our first Tensor Core GPU in 2017, built from the ground-up for the new era of AI, and our first autonomousdriving system-on-chips, or SoC, in 2018. Our acquisition of Mellanox in 2020 expanded our innovation canvas to include networking and led to the introductionof a new processor class – the data processing unit, or DPU. Over the past 5 years, we have built full software stacks that run on top of our GPUs and CUDA to\nbring AI to the world’s largest industries, including NVIDIA DRIVE stack for autonomous driving, Clara for healthcare, and Omniverse for industrial digitalization;and introduced the NVIDIA AI Enterprise software – essentially an operating system for enterprise AI applications. In 2023, we introduced our first data center..."
1,NVIDIA_Report_2024.pdf,33,0.8847,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
2,NVIDIA_Report_2024.pdf,11,0.6264,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.4720,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.6063,"growing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory provisions as a percentage of revenue.\nOperating expenses increased for fiscal year 2024, driven by growth in employees and compensation increases. Fiscal year 2023 also included a $1.4 billion\nacquisition termination charge related to the proposed Arm transaction.\nMarket Platform Highlights\nData Center revenue for fiscal year 2024 was $47.5 billion, up 217% from fiscal year 2023. In Data Center, we launched AI inference platforms that combine our..."
1,NVIDIA_Report_2024.pdf,35,0.5620,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,35,0.4688,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
3,NVIDIA_Report_2024.pdf,35,0.3610,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,5,0.6061,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators...."
1,NVIDIA_Report_2024.pdf,6,0.5948,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,3,0.5665,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
3,NVIDIA_Report_2024.pdf,4,0.4473,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."


# Complete Evaluation

In [31]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="HyDE",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'HyDE-d9a7c634' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=d9546220-c3eb-41ec-b645-dc456f097dcf




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,41,0.8791,"Table of Contents\nLiquidity and Capital Resources\n Jan 28, 2024 Jan 29, 2023\n (In millions)\nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nCash, cash equivalents, and marketable securities $ 25,984 $ 13,296 \n Year Ended\nJan 28, 2024 Jan 29, 2023\n (In millions)\nNet cash provided by operating activities $ 28,090 $ 5,641 \nNet cash provided by (used in) investing activities $ (10,566)$ 7,375 \nNet cash used in financing activities $ (13,633)$ (11,617)\nOur investment policy requires the purchase of highly rated fixed income securities, the diversification of investment types and credit exposures, and certainmaturity limits on our portfolio...."
1,NVIDIA_Report_2024.pdf,41,0.8273,"Liquidity\nOur primary sources of liquidity are our cash, cash equivalents, and marketable securities, and the cash generated by our operations. At the end of fiscal year\n2024, we had $26.0 billion in cash, cash equivalents and marketable securities. We believe that we have sufficient liquidity to meet our operating requirementsfor at least the next twelve months, and for the foreseeable future, including our future supply obligations and $1.3 billion of debt repayment due in fiscal year2025 and share purchases. We continuously evaluate our liquidity and capital resources, including our access to external capital, to ensure we can finance\nfuture capital requirements...."
2,NVIDIA_Report_2024.pdf,53,0.7871,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Statements of Cash Flows\n(In millions)\nYear Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nCash flows from operating activities: \nNet income $ 29,760 $ 4,368 $ 9,752 \nAdjustments to reconcile net income to net cash provided by operating activities: \nStock-based compensation expense 3,549 2,709 2,004 \nDepreciation and amortization 1,508 1,544 1,174 \nDeferred income taxes (2,489) (2,164) (406)\n(Gains) losses on investments in non-affiliated entities, net (238) 45 (100)\nAcquisition termination cost — 1,353 — \nOther (278) (7) 47 \nChanges in operating assets and liabilities, net of acquisitions: \nAccounts receivable (6,172) 822 (2,215)\nInventories (98) (2,554) (774)\nPrepaid expenses and other assets (1,522) (1,517) (1,715)..."
3,NVIDIA_Report_2024.pdf,51,0.7535,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Balance Sheets(In millions, except par value)\n Jan 28, 2024 Jan 29, 2023\nAssets \nCurrent assets: \nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nAccounts receivable, net 9,999 3,827 \nInventories 5,282 5,159 \nPrepaid expenses and other current assets 3,080 791 \nTotal current assets 44,345 23,073 \nProperty and equipment, net 3,914 3,807 \nOperating lease assets 1,346 1,038 \nGoodwill 4,430 4,372 \nIntangible assets, net 1,112 1,676 \nDeferred income tax assets 6,081 3,396 \nOther assets 4,500 3,820 \nTotal assets $ 65,728 $ 41,182 \nLiabilities and Shareholders' Equity\nCurrent liabilities: \nAccounts payable $ 2,699 $ 1,193 \nAccrued and other current liabilities 6,682 4,120..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.6199,"full-stack inference software with NVIDIA Ada, NVIDIA Hopper and NVIDIA Grace Hopper processors optimized for generative AI, LLMs and other AI workloads.We introduced NVIDIA DGX Cloud and AI Foundations to help businesses create and operate custom large language models and generative AI models. As AV\nalgorithms move to video transformers, and more cars are equipped with cameras, we expect NVIDIA’s automotive data center processing demand to growsignificantly. We estimate that in fiscal year 2024, approximately 40% of Data Center revenue was for AI inference. In the fourth quarter of fiscal year 2024, largecloud providers represented more than half of our Data Center revenue, supporting both internal workloads and external customers. We announced NVIDIA..."
1,NVIDIA_Report_2024.pdf,5,0.6090,"Table of Contents\nAt the foundation of the NVIDIA accelerated computing platform are our GPUs, which excel at parallel workloads such as the training and inferencing of neural\nnetworks. They are available in the NVIDIA accelerated computing platform and in industry standard servers from every major cloud provider and server maker.Beyond GPUs, our data center platform expanded to include DPUs in fiscal year 2022 and CPUs in fiscal year 2024. We can optimize across the entire\ncomputing, networking and storage stack to deliver data center-scale computing solutions...."
2,NVIDIA_Report_2024.pdf,6,0.5975,"compute solutions include all three major processing units in AI servers – GPUs, CPUs, and DPUs. GPUs are uniquely suited to AI, and we will continue to addAI-specific features to our GPU architecture to further extend our leadership position. In addition, we offer DGX Cloud, an AI-training-as-a-service platform, andNeMo – a complete solution for building enterprise-ready Large Language Models, or LLMs, using open source and proprietary LLMs created by NVIDIA and\nthird parties. Our AI technology leadership is reinforced by our large and expanding ecosystem in a virtuous cycle. Our computing platforms are available fromvirtually every major server maker and CSP, as well as on our own AI supercomputers. There are over 4.7 million developers worldwide using CUDA and our..."
3,NVIDIA_Report_2024.pdf,6,0.5782,"this platform approach in each of our target markets.\nExtending our technology and platform leadership in AI. We provide a complete, end-to-end accelerated computing platform for AI, addressing both trainingand inferencing. This includes full-stack data center-scale compute and networking solutions across processing units, interconnects, systems, and software. Our..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,41,0.7411,"Cash provided by operating activities increased in fiscal year 2024 compared to fiscal year 2023, due to growth in revenue. Accounts receivable balance in fiscalyear 2024 reflected $557 million from customer payments received ahead of the invoice due date.\nCash provided by investing activities decreased in fiscal year 2024 compared to fiscal year 2023, primarily driven by lower marketable securities maturities and\nhigher purchases of marketable securities.\nCash used in financing activities increased in fiscal year 2024 compared to fiscal year 2023, due to a debt repayment and higher tax payments related to RSUs,partially offset by lower share repurchases.\nLiquidity..."
1,NVIDIA_Report_2024.pdf,35,0.7261,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
2,NVIDIA_Report_2024.pdf,31,0.6873,"legal requirements, and other factors. The program does not obligate NVIDIA to acquire any particular amount of common stock and the program may besuspended at any time at our discretion.\nIn fiscal year 2024, we paid $395 million in quarterly cash dividends. Our cash dividend program and the payment of future cash dividends under that program\nare subject to our Board of Directors' continuing determination that the dividend program and the declaration of dividends thereunder are in the best interests ofour shareholders.\nThe following table presents details of our share repurchase transactions during the fourth quarter of fiscal year 2024:\nPeriod\nTotal Numberof SharesPurchased(In millions) Average Price Paidper Share..."
3,NVIDIA_Report_2024.pdf,53,0.6219,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Statements of Cash Flows\n(In millions)\nYear Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nCash flows from operating activities: \nNet income $ 29,760 $ 4,368 $ 9,752 \nAdjustments to reconcile net income to net cash provided by operating activities: \nStock-based compensation expense 3,549 2,709 2,004 \nDepreciation and amortization 1,508 1,544 1,174 \nDeferred income taxes (2,489) (2,164) (406)\n(Gains) losses on investments in non-affiliated entities, net (238) 45 (100)\nAcquisition termination cost — 1,353 — \nOther (278) (7) 47 \nChanges in operating assets and liabilities, net of acquisitions: \nAccounts receivable (6,172) 822 (2,215)\nInventories (98) (2,554) (774)\nPrepaid expenses and other assets (1,522) (1,517) (1,715)..."


...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,33,0.8906,"Headquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nRecent Developments, Future Objectives and Challenges\nDemand and Supply, Product Transitions, and New Products and Business Models\nDemand for our data center systems and products surged in fiscal year 2024. Entering fiscal year 2025, we are gathering customer demand indications acrossseveral product transitions. We have demand visibility for our new data center products ramping later in fiscal year 2025. We have increased our supply andcapacity purchases with existing suppliers, added new vendors and entered into prepaid manufacturing and capacity agreements. These increased purchase..."
1,NVIDIA_Report_2024.pdf,33,0.8427,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
2,NVIDIA_Report_2024.pdf,11,0.6384,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.5220,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,6,0.5875,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
1,NVIDIA_Report_2024.pdf,3,0.5760,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,5,0.5755,"In fiscal year 2024, we launched the NVIDIA DGX Cloud, an AI-training-as-a-service platform which includes cloud-based infrastructure and software for AI,customizable pretrained AI models, and access to NVIDIA experts. We have partnered with leading cloud service providers to host this service in their datacenters.\nGaming\nGaming is the largest entertainment industry, with PC gaming as the predominant platform. Many factors propel its growth, including new high production valuegames and franchises, the continued rise of competitive gaming, or eSports, social connectivity and the increasing popularity of game streamers, modders, or\ngamers who remaster games, and creators...."
3,NVIDIA_Report_2024.pdf,4,0.4613,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,0.7809,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
1,NVIDIA_Report_2024.pdf,41,0.7640,"Liquidity\nOur primary sources of liquidity are our cash, cash equivalents, and marketable securities, and the cash generated by our operations. At the end of fiscal year\n2024, we had $26.0 billion in cash, cash equivalents and marketable securities. We believe that we have sufficient liquidity to meet our operating requirementsfor at least the next twelve months, and for the foreseeable future, including our future supply obligations and $1.3 billion of debt repayment due in fiscal year2025 and share purchases. We continuously evaluate our liquidity and capital resources, including our access to external capital, to ensure we can finance\nfuture capital requirements...."
2,NVIDIA_Report_2024.pdf,53,0.6362,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Statements of Cash Flows\n(In millions)\nYear Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nCash flows from operating activities: \nNet income $ 29,760 $ 4,368 $ 9,752 \nAdjustments to reconcile net income to net cash provided by operating activities: \nStock-based compensation expense 3,549 2,709 2,004 \nDepreciation and amortization 1,508 1,544 1,174 \nDeferred income taxes (2,489) (2,164) (406)\n(Gains) losses on investments in non-affiliated entities, net (238) 45 (100)\nAcquisition termination cost — 1,353 — \nOther (278) (7) 47 \nChanges in operating assets and liabilities, net of acquisitions: \nAccounts receivable (6,172) 822 (2,215)\nInventories (98) (2,554) (774)\nPrepaid expenses and other assets (1,522) (1,517) (1,715)..."
3,NVIDIA_Report_2024.pdf,51,0.6351,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Balance Sheets(In millions, except par value)\n Jan 28, 2024 Jan 29, 2023\nAssets \nCurrent assets: \nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nAccounts receivable, net 9,999 3,827 \nInventories 5,282 5,159 \nPrepaid expenses and other current assets 3,080 791 \nTotal current assets 44,345 23,073 \nProperty and equipment, net 3,914 3,807 \nOperating lease assets 1,346 1,038 \nGoodwill 4,430 4,372 \nIntangible assets, net 1,112 1,676 \nDeferred income tax assets 6,081 3,396 \nOther assets 4,500 3,820 \nTotal assets $ 65,728 $ 41,182 \nLiabilities and Shareholders' Equity\nCurrent liabilities: \nAccounts payable $ 2,699 $ 1,193 \nAccrued and other current liabilities 6,682 4,120..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,0.9020,"“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request...."
1,NVIDIA_Report_2024.pdf,3,0.9002,"marking the “Big Bang” moment of AI. We introduced our first Tensor Core GPU in 2017, built from the ground-up for the new era of AI, and our first autonomousdriving system-on-chips, or SoC, in 2018. Our acquisition of Mellanox in 2020 expanded our innovation canvas to include networking and led to the introductionof a new processor class – the data processing unit, or DPU. Over the past 5 years, we have built full software stacks that run on top of our GPUs and CUDA to\nbring AI to the world’s largest industries, including NVIDIA DRIVE stack for autonomous driving, Clara for healthcare, and Omniverse for industrial digitalization;and introduced the NVIDIA AI Enterprise software – essentially an operating system for enterprise AI applications. In 2023, we introduced our first data center..."
2,NVIDIA_Report_2024.pdf,11,0.6197,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,11,0.4834,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,3,0.9737,"marking the “Big Bang” moment of AI. We introduced our first Tensor Core GPU in 2017, built from the ground-up for the new era of AI, and our first autonomousdriving system-on-chips, or SoC, in 2018. Our acquisition of Mellanox in 2020 expanded our innovation canvas to include networking and led to the introductionof a new processor class – the data processing unit, or DPU. Over the past 5 years, we have built full software stacks that run on top of our GPUs and CUDA to\nbring AI to the world’s largest industries, including NVIDIA DRIVE stack for autonomous driving, Clara for healthcare, and Omniverse for industrial digitalization;and introduced the NVIDIA AI Enterprise software – essentially an operating system for enterprise AI applications. In 2023, we introduced our first data center..."
1,NVIDIA_Report_2024.pdf,12,0.9426,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
2,NVIDIA_Report_2024.pdf,11,0.8141,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
3,NVIDIA_Report_2024.pdf,11,0.7269,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
